In [1]:
!pip install --quiet playwright

Python wurde nicht gefunden; ohne Argumente ausführen, um aus dem Microsoft Store zu installieren, oder deaktivieren Sie diese Verknüpfung unter "Einstellungen > Apps > Erweiterte App-Einstellungen > App-Ausführungsaliase".


In [2]:
!C:\Users\ralfb\anaconda3\Scripts\ipython.exe -m playwright install chromium

|                                                                                |   0% of 148.9 MiB
|â– â– â– â– â– â– â– â–                                                                         |  10% of 148.9 MiB
|â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â–                                                                 |  20% of 148.9 MiB
|â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â–                                                         |  30% of 148.9 MiB
|â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â–                                                 |  40% of 148.9 MiB
|â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â–                                         |  50% of 148.9 MiB
|â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â– â–          

In [3]:
import asyncio
from pathlib import Path
from typing import List

from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeoutError

CHATGPT_URL = "https://chatgpt.com/"
USER_DATA_DIR = str(Path.home() / ".chatgpt-playwright-profile")

# Heuristische Selektoren für die Chat-Liste und Aktionen im Drei-Punkte-Menü
SIDEBAR_SELECTOR_CANDIDATES = [
    '[data-testid="sidebar"]',
    'nav[aria-label*="history" i]',
    'nav[aria-label*="chat" i]'
]
CHAT_ROW_CANDIDATES = [
    '[data-testid="conversation-item"]',
    '[data-testid="history-item"]',
    'a[href*="/c/"]',
    'div[role="listitem"] a'
]
ROW_MENU_BUTTON_CANDIDATES = [
    '[aria-label="More options"]',
    'button[aria-label*="more" i]',
    'button:has-text("⋯")',
    'button[title*="mehr" i]'
]
MENU_DELETE_CANDIDATES = [
    'role=menuitem[name=/delete/i]',
    'text=/^Delete$/i',
    'text=/^Löschen$/i'
]
CONFIRM_DELETE_CANDIDATES = [
    'role=button[name=/delete/i]',
    'role=button[name=/confirm/i]',
    'button:has-text("Delete")',
    'button:has-text("Löschen")'
]
HOME_BUTTON_CANDIDATES = [
    'a:has-text("Home")',
    'button:has-text("Home")',
    '[data-testid="home-nav"]'
]

async def ensure_logged_in(page):
    await page.goto(CHATGPT_URL, wait_until="load")
    # Warten bis entweder die Chat-UI erscheint oder der Login-Button sichtbar ist
    try:
        await page.wait_for_selector('text=/New chat|Neuer Chat/i', timeout=8000)
        return
    except PlaywrightTimeoutError:
        pass

    # Falls nicht eingeloggt: Anwendern Zeit geben, sich einzuloggen
    print("Bitte logge dich im geöffneten Browser bei ChatGPT ein, dann kehre zurück.")
    print("Sobald die Chat-Seitenleiste sichtbar ist, drücke hier Enter …")
    input()

async def click_first_that_exists(page, selectors: List[str], within=None, timeout=4000):
    for sel in selectors:
        try:
            handle = (within or page).locator(sel).first
            await handle.wait_for(state="visible", timeout=timeout)
            return handle
        except PlaywrightTimeoutError:
            continue
    return None

async def go_to_home_if_possible(page):
    btn = await click_first_that_exists(page, HOME_BUTTON_CANDIDATES)
    if btn:
        try:
            await btn.click()
        except:
            pass

async def delete_visible_rows(page) -> int:
    # Seitenleiste finden
    sidebar = await click_first_that_exists(page, SIDEBAR_SELECTOR_CANDIDATES, timeout=6000)
    if not sidebar:
        print("⚠️ Konnte die Chat-Seitenleiste nicht finden.")
        return 0

    deleted = 0

    # Endlosschleife: immer oberstes sichtbares Item löschen, bis keins mehr da ist
    while True:
        row = None
        for row_sel in CHAT_ROW_CANDIDATES:
            rows = sidebar.locator(row_sel)
            if await rows.count() > 0:
                # Nimm das erste sichtbare Element
                # (nicht das gerade aktive, das ggf. kein Menü hat)
                for i in range(await rows.count()):
                    candidate = rows.nth(i)
                    if await candidate.is_visible():
                        row = candidate
                        break
            if row:
                break

        if not row:
            break  # nichts mehr sichtbar

        # Menü-Button im Row-Kontext suchen
        menu_btn = None
        for msel in ROW_MENU_BUTTON_CANDIDATES:
            try:
                mb = row.locator(msel).first
                if await mb.count() > 0 and await mb.is_enabled():
                    menu_btn = mb
                    break
            except:
                continue

        if not menu_btn:
            # versuche: Hover -> Kontextmenü erscheint
            try:
                await row.hover()
                menu_btn = await click_first_that_exists(page, ROW_MENU_BUTTON_CANDIDATES, within=row, timeout=1500)
            except:
                pass

        if not menu_btn:
            # Wenn kein Menü gefunden wird, scrolle die Sidebar ein Stück und versuche das nächste Item
            await sidebar.evaluate("(el) => el.scrollBy(0, 200)", force_expr=True)
            continue

        # Menü öffnen
        try:
            await menu_btn.click()
        except:
            # nächstes Item probieren
            await sidebar.evaluate("(el) => el.scrollBy(0, 200)", force_expr=True)
            continue

        # „Delete/Löschen“ im Menü
        delete_item = await click_first_that_exists(page, MENU_DELETE_CANDIDATES, timeout=2000)
        if not delete_item:
            # Menü schließen (Esc) und weiter
            await page.keyboard.press("Escape")
            await sidebar.evaluate("(el) => el.scrollBy(0, 200)", force_expr=True)
            continue

        await delete_item.click()

        # Bestätigen
        confirm_btn = await click_first_that_exists(page, CONFIRM_DELETE_CANDIDATES, timeout=4000)
        if confirm_btn:
            await confirm_btn.click()
            deleted += 1
            # kleinen Moment warten bis Eintrag verschwindet
            await page.wait_for_timeout(500)
        else:
            # Falls kein Bestätigungsdialog (z. B. anderes UI): weiter
            deleted += 1

        # etwas scrollen, um neue Einträge zu laden
        await sidebar.evaluate("(el) => el.scrollBy(0, 150)", force_expr=True)

    return deleted

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch_persistent_context(USER_DATA_DIR, headless=False)
        page = await browser.new_page()
        await ensure_logged_in(page)

        # Optional: in „Home“ wechseln, damit nur Nicht-Projekt-Chats sichtbar sind
        await go_to_home_if_possible(page)

        total_deleted = 0
        # In Batches arbeiten, bis nichts mehr da ist
        for _ in range(50):  # Sicherheitsgrenze
            deleted = await delete_visible_rows(page)
            total_deleted += deleted
            if deleted == 0:
                break

        print(f"✅ Fertig. Gelöschte Chats: {total_deleted}")
        # Browser offen lassen, damit du das Ergebnis siehst
        input("Drücke Enter zum Schließen…")
        await browser.close()

if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop